<a href="https://colab.research.google.com/github/Zenith1618/Learn-PySpark/blob/main/6_ML_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Missing').getOrCreate()

# Read the dataset

In [5]:
training = spark.read.csv('test1.csv', header = True, inferSchema = True)

In [6]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [8]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
training.columns

['Name', 'age', 'Experience', 'Salary']

# PySpark Preprocessing

First we create a grouped columns of independant columns


[Age, Experience] ---> new feature ---> Independant feature

In [10]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['age', 'Experience'], outputCol="Independent Features")

In [11]:
output = assembler.transform(training)

output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [12]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [13]:
finalized_data = output.select("Independent Features", "Salary")

In [15]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



# SparkML

In [22]:
from pyspark.ml.regression import LinearRegression

# Train Test Split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [23]:
# Coefficients
regressor.coefficients

DenseVector([-428.5714, 1928.5714])

In [24]:
# Intercepts
regressor.intercept

23999.999999999996

In [25]:
# Prediction
pred_results = regressor.evaluate(test_data)

In [26]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16928.571428571428|
|          [24.0,3.0]| 20000|           19500.0|
|          [29.0,4.0]| 20000|19285.714285714286|
|          [30.0,8.0]| 25000|26571.428571428572|
+--------------------+------+------------------+



In [27]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1178.5714285714284, 1737244.8979591837)